In [10]:
import tflearn
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import preprocessing
import chess_game as cg
import time

In [39]:
board = tflearn.layers.core.input_data(shape=(None, 64, 8), name='board_input')
aux = tflearn.layers.core.input_data(shape=(None, 7), name='aux_input')
Y = tflearn.layers.core.input_data(shape=(None, 3), name='labels')

board_input_reshaped = tf.reshape(board, [-1, 8, 8, 8])

conv4 = tflearn.conv_2d(board_input_reshaped, 32, 4, activation='relu')


conv2 = tflearn.conv_2d(board_input_reshaped, 32, 2, activation='relu')
conv2_4 = tflearn.conv_2d(conv2, 64, 4, activation='relu')
# note: this is how to get separate branches of a larger NN and then have them converge
fc1 = tflearn.layers.core.fully_connected(tf.concat([conv4, conv2_4], 3), n_units=1000, activation='relu')

fc2 = tflearn.layers.core.fully_connected(tf.concat([fc1, aux], 1), n_units=2000, activation='relu')

net = tflearn.layers.core.fully_connected(fc2, n_units=3, activation='relu')

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(loss)

init = tf.global_variables_initializer()
#net = tflearn.regression(net, optimizer='adam', metric=loss,
#                         loss='categorical_crossentropy')
#model = tflearn.DNN(net, tensorboard_verbose=0)


In [18]:
a = pd.read_csv('board_positions/board_positions_1.csv')
a = a[~a.isnull().any(axis=1)]
a = a[a['moves_in_game'] - a['move_num']<10]
le = preprocessing.LabelEncoder()
le.fit(['k', 'q', 'r', 'b', 'n', 'p', ' ', 'P', 'N', 'B', 'R', 'Q', 'K'])

bcols = a.columns[:64]

a[bcols] = le.transform(a[bcols])
    

In [36]:
def get_training_data(data):
    
    d = data.sample(100)
    train = d[d.columns[:71]]
    board = train[train.columns[:64]]
    aux = train[train.columns[64:]]
    board_onehot = np.zeros((100, 64, 8))
    board_onehot[:, :, 0] = board.isin([7, 8, 9, 10, 11, 12])
    board_onehot[:, :, 1] = board.isin([0, 1, 2, 3, 4, 5])
    board_onehot[:, :, 2] = board.isin([0, 12])
    board_onehot[:, :, 3] = board.isin([1, 11])
    board_onehot[:, :, 4] = board.isin([2, 10])
    board_onehot[:, :, 5] = board.isin([3, 9])
    board_onehot[:, :, 6] = board.isin([4, 8])
    board_onehot[:, :, 7] = board.isin([5, 7])
    labels = np.zeros((100,3))
    labels[d['winner']==-1, 0] = 1
    labels[d['winner']==0, 1] = 1
    labels[d['winner']==1, 2] = 1
    
    return board_onehot, aux, labels
    

In [40]:
sess = tf.Session()
sess.run(init)

batch_size = 128
for epoch in range(10):  # 2 epochs
    for i in range(200):
        batch_board, batch_aux, batch_ys = get_training_data(a)
        sess.run(optimizer, feed_dict={board: batch_board, aux: batch_aux, Y: batch_ys})
        cost = sess.run(loss, feed_dict={board: batch_board, aux: batch_aux, Y: batch_ys})

        if i % 20 == 0:
            print("Epoch:", '%03d' % (epoch + 1), "Step:", '%03d' % i,
                  "Loss:", str(cost))

Epoch: 001 Step: 000 Loss: 1.0957798
Epoch: 001 Step: 020 Loss: 1.0986392
Epoch: 001 Step: 040 Loss: 1.0973583
Epoch: 001 Step: 060 Loss: 1.0955995
Epoch: 001 Step: 080 Loss: 1.0982157
Epoch: 001 Step: 100 Loss: 1.0945449
Epoch: 001 Step: 120 Loss: 1.0985711
Epoch: 001 Step: 140 Loss: 1.0984702
Epoch: 001 Step: 160 Loss: 1.0954676
Epoch: 001 Step: 180 Loss: 1.087503
Epoch: 002 Step: 000 Loss: 1.0984743
Epoch: 002 Step: 020 Loss: 1.0995202
Epoch: 002 Step: 040 Loss: 1.0930213


KeyboardInterrupt: 

In [56]:
b = cg.game_board()
next_positions = b.find_all_next_board_positions('W')
next_positions_ar = np.array([ele.to_csv_format() for ele in next_positions])
next_positions_ar[:, :64] = le.transform(next_positions_ar[:, :64])


In [92]:
next_positions = b.find_all_next_board_positions('W')
next_positions_ar = np.array([ele.to_csv_format() for ele in next_positions])
next_positions_ar[:, :64] = le.transform(next_positions_ar[:, :64])

next_scores = sess.run(net, feed_dict={X:next_positions_ar})
b = next_positions[np.argmax(next_scores[:,2])]

In [93]:
print(b)

| r | n | b |   | k | b |   | r
| p | p | p | p |   | p |   | p
|   |   |   |   |   |   |   |  
|   |   |   |   | q |   |   |  
|   |   |   |   | P | p | P |  
|   |   |   |   |   |   |   |  
| P | P | P |   |   |   | B | P
| R | N |   | K |   |   | N | R


In [90]:
b.move('Qxe5')

In [91]:
print(b)

| r | n | b |   | k | b |   | r
| p | p | p | p |   | p |   | p
|   |   |   |   |   |   |   |  
|   |   |   |   | q |   |   |  
|   |   |   |   | P | p | P |  
|   |   |   |   |   |   |   |  
| P | P | P |   |   |   |   | P
| R | N |   | K |   | B | N | R


In [55]:
import importlib
importlib.reload(cg)

<module 'chess_game' from '/Users/pschale/pythonstuff/chess_ai_project/chess_game.py'>

In [4]:
a = pd.read_csv('board_positions/board_positions_1.txt')

In [11]:
a[a['moves_in_game'] - a['move_num']<10][a.columns[:71]]

,a1,a2,a3,a4,a5,a6,a7,a8,b1,b2,...,h6,h7,h8,white_tomove,w_castle_kingside,w_castle_queenside,b_castle_kingside,b_castle_queenside,w_material,b_material
50,,P,,,,,p,,R,,...,,p,,0,0,0,0,0,15,16
51,,P,,,,,p,,R,,...,,p,,1,0,0,0,0,12,16
52,,P,,,,,p,,,,...,,p,,0,0,0,0,0,12,16
53,,P,,,,,p,,,,...,,p,,1,0,0,0,0,12,16
54,,P,,,,,p,,,,...,,p,,0,0,0,0,0,12,16
55,,P,,,,,p,,,,...,,,,1,0,0,0,0,12,16
56,,P,,,,,p,,,,...,,,,0,0,0,0,0,12,16
57,,P,,,,,p,,,,...,,,,1,0,0,0,0,11,16
58,,P,,,,,p,,,,...,,,,0,0,0,0,0,11,15
59,,r,,,,,p,,,,...,,,,1,0,0,0,0,10,15


In [ ]:
tf.__version__

In [2]:
a = pd.read_csv('board_positions/board_positions_1.csv')
a = a[~a.isnull().any(axis=1)]
a = a[a['moves_in_game'] - a['move_num']<10]
le = preprocessing.LabelEncoder()
le.fit(['k', 'q', 'r', 'b', 'n', 'p', ' ', 'P', 'N', 'B', 'R', 'Q', 'K'])

bcols = a.columns[:64]

a[bcols] = le.transform(a[bcols])

In [3]:
c = a.sample(100)

In [27]:
c.shape

(100, 75)

In [31]:
c.as_matrix()[:, :64]

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  4, ...,  0,  0,  0],
       [ 0,  4,  0, ...,  0,  0,  8],
       ...,
       [ 0,  4, 12, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  4,  0, ...,  0, 10,  0]])

In [15]:
start = time.time()
train = c[c.columns[:71]]
train_onehot = np.zeros((100, 71, 8))
train_onehot[:, :, 0] = train.isin([7, 8, 9, 10, 11, 12])
train_onehot[:, :, 1] = train.isin([0, 1, 2, 3, 4, 5])
train_onehot[:, :, 2] = train.isin([0, 12])
train_onehot[:, :, 3] = train.isin([1, 11])
train_onehot[:, :, 4] = train.isin([2, 10])
train_onehot[:, :, 5] = train.isin([3, 9])
train_onehot[:, :, 6] = train.isin([4, 8])
train_onehot[:, :, 7] = train.isin([5, 7])
end = time.time()
print(end - start)

0.010066986083984375


In [13]:
train_onehot[:, :, 0]

array([[0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])